In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import os
import json

In [38]:
reviews_healthstory_path = "/content/drive/MyDrive/Master/Anul 1/BioNLP/dataset/reviews/HealthStory.json"
reviews_healthrelease_path = "/content/drive/MyDrive/Master/Anul 1/BioNLP/dataset/reviews/HealthRelease.json"
healthstory_df = pd.read_json(reviews_healthstory_path)
healthrelease_df = pd.read_json(reviews_healthrelease_path)

In [29]:
reviews_healthstory_content_path = "/content/drive/MyDrive/Master/Anul 1/BioNLP/dataset/content/HealthStory/"
reviews_healthrelease_content_path = "/content/drive/MyDrive/Master/Anul 1/BioNLP/dataset/content/HealthRelease/"

In [31]:
healthstory_data = []

# Iterate through all files in the directory
for filename in os.listdir(reviews_healthstory_content_path):
    if filename.endswith(".json"):
        file_path = os.path.join(reviews_healthstory_content_path, filename)
        with open(file_path, "r", encoding="utf-8") as file:
            try:
                data = json.load(file)
                # Extract 'text', 'title', and use 'filename' as 'id'
                healthstory_data.append({
                    "id": filename[:-5],  # Remove ".json" extension
                    "title": data["title"],
                    "text": data["text"]
                })
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")


In [40]:
healthstory_content_df = pd.DataFrame(healthstory_data)
healthstory_content_df = healthstory_content_df.merge(healthrelease_df[['news_id', 'rating']], left_on='id', right_on='news_id', how='left')

# Drop the redundant 'news_id' column after merge
healthstory_content_df.drop(columns=['news_id'], inplace=True)

In [42]:
csv_path = "/content/drive/MyDrive/Master/Anul 1/BioNLP/dataset/healthstory_content.csv"
healthstory_content_df.to_csv(csv_path, index=False)

In [45]:
healthstory_content_df

,id,title,text,rating
0,story_reviews_00000,Virtual reality to help detect early risk of A...,Scientists have found an unexpected use for vi...,1
1,story_reviews_01699,Studies: Bone drugs may help prevent breast ca...,FORUM: LIVING WITH CANCER FORUM: LIVING WITH C...,4
2,story_reviews_01270,Vaginal steam bath finds a place among Souther...,Vaginal steam bath finds a place among Souther...,4
3,story_reviews_00897,New Procedure Makes Hyperhidrosis No Sweat,"March 19, 2012 (San Diego) -- Ultrasound treat...",2
4,story_reviews_01484,Could Drinking Help Thwart Rheumatoid Arthritis?,By Amanda Gardner\n\nHealthDay Reporter\n\nWED...,2
...,...,...,...,...
1633,story_reviews_01059,Who should get screened for lung cancer?,NEW YORK (Reuters Health) - Using chest scans ...,5
1634,story_reviews_01303,2 Treatments for Retina Diseases Make Progress,But the drug works best when given every four ...,3
1635,story_reviews_01442,Biosynthetic Corneas Show Promise in Transplants,By Randy Dotinga\n\nHealthDay Reporter\n\nWEDN...,4
1636,story_reviews_00215,Acupuncture May Be Effective Painkiller in the ER,"En Español\n\nMONDAY, June 19, 2017 (HealthDay...",2


In [47]:
healthrelease_data = []

# Iterate through all files in the directory
for filename in os.listdir(reviews_healthrelease_content_path):
    if filename.endswith(".json"):
        file_path = os.path.join(reviews_healthrelease_content_path, filename)
        with open(file_path, "r", encoding="utf-8") as file:
            try:
                data = json.load(file)
                # Extract 'text', 'title', and use 'filename' as 'id'
                healthrelease_data.append({
                    "id": filename[:-5],  # Remove ".json" extension
                    "title": data["title"],
                    "text": data["text"]
                })
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")

healthrelease_content_df = pd.DataFrame(healthrelease_data)
healthrelease_content_df = healthrelease_content_df.merge(healthrelease_df[['news_id', 'rating']], left_on='id', right_on='news_id', how='left')

# Drop the redundant 'news_id' column after merge
healthrelease_content_df.drop(columns=['news_id'], inplace=True)

In [49]:
csv_path = "/content/drive/MyDrive/Master/Anul 1/BioNLP/dataset/healthrelease_content.csv"
healthrelease_content_df.to_csv(csv_path, index=False)

In [11]:
import pandas as pd
import matplotlib.pyplot as plt


# Define a threshold for classification
threshold = 3

# Create a label column based on the threshold (1 = fake, 0 = true)
healthstory_df['label'] = healthstory_df['rating'].apply(lambda x: 1 if threshold > x else 0)
healthrelease_df['label'] = healthrelease_df['rating'].apply(lambda x: 1 if threshold > x else 0)

def plot_and_save(df, title, filename):
    plt.figure(figsize=(8, 4))
    df['rating'].value_counts().sort_index().plot(kind='bar')
    plt.title(title)
    plt.xlabel('Rating')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

# Plot and save the rating distribution for HealthStory
plot_and_save(healthstory_df, 'Rating Distribution - HealthStory', 'healthstory_rating_distribution.pdf')

# Plot and save the rating distribution for HealthRelease
plot_and_save(healthrelease_df, 'Rating Distribution - HealthRelease', 'healthrelease_rating_distribution.pdf')

In [14]:
from wordcloud import WordCloud

def generate_and_save_wordcloud(df, label, filename):
    news_titles = df[df['label'] == label]['original_title']
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(news_titles))
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'{"Fake" if label == 1 else "Real"} News Word Cloud - Titles - HealthStory')
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

generate_and_save_wordcloud(healthstory_df, 1, 'fake_news_wordcloud.pdf')

generate_and_save_wordcloud(healthstory_df, 0, 'real_news_wordcloud.pdf')

In [15]:
generate_and_save_wordcloud(healthrelease_df, 1, 'fake_news_wordcloud_health_release.pdf')

generate_and_save_wordcloud(healthrelease_df, 0, 'real_news_wordcloud_health_release.pdf')

In [18]:
from collections import Counter

tags = sum(healthstory_df['tags'], [])

tag_counts = Counter(tags)

top_10_tags = tag_counts.most_common(10)

print("Top 10 Tags:")
for tag, count in top_10_tags:
    print(f"{tag}: {count}")

Top 10 Tags:
breast cancer: 65
prostate cancer: 45
weight loss: 37
surgery: 27
cholesterol: 25
colon cancer: 24
melanoma: 23
aspirin: 22
immunotherapy: 21
Pregnancy: 21


In [19]:
from collections import Counter

tags = sum(healthrelease_df['tags'], [])

tag_counts = Counter(tags)

top_10_tags = tag_counts.most_common(10)

print("Top 10 Tags:")
for tag, count in top_10_tags:
    print(f"{tag}: {count}")

Top 10 Tags:
prostate cancer: 14
blood test: 13
weight loss: 12
depression: 12
FDA: 11
immunotherapy: 10
concussion: 10
JAMA: 9
obesity: 8
heart disease: 8


In [52]:
merged_df = pd.concat([healthrelease_content_df, healthstory_content_df], ignore_index=True)

merged_df['label'] = merged_df['rating'].apply(lambda x: 1 if x < 3 else 0)

csv_path = "/content/drive/MyDrive/Master/Anul 1/BioNLP/dataset/merged_health_content.csv"
merged_df.to_csv(csv_path, index=False)


In [53]:
merged_df

,id,title,text,rating,label
0,news_reviews_00000,Tiny implantable device short-circuits hunger ...,"MADISON, Wis. -- More than 700 million adults ...",2,1
1,news_reviews_00563,New Study Published on Cover of Urology Finds ...,"SAN DIEGO, July 14, 2015 /PRNewswire/ -- Genom...",3,0
2,news_reviews_00114,Device Keeps Donor Kidneys Healthy Outside the...,"Newswise — Toronto, ON (February 5, 2018) -- A...",2,1
3,news_reviews_00079,New research ranks the effectiveness of nonsur...,"ROSEMONT, Ill. (May 1, 2018)--An estimated 45 ...",3,0
4,news_reviews_00179,New recommendations for managing menopausal sy...,WASHINGTON -- A large proportion of the world'...,1,1
...,...,...,...,...,...
2232,story_reviews_01059,Who should get screened for lung cancer?,NEW YORK (Reuters Health) - Using chest scans ...,5,0
2233,story_reviews_01303,2 Treatments for Retina Diseases Make Progress,But the drug works best when given every four ...,3,0
2234,story_reviews_01442,Biosynthetic Corneas Show Promise in Transplants,By Randy Dotinga\n\nHealthDay Reporter\n\nWEDN...,4,0
2235,story_reviews_00215,Acupuncture May Be Effective Painkiller in the ER,"En Español\n\nMONDAY, June 19, 2017 (HealthDay...",2,1


In [55]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(merged_df, test_size=0.4, random_state=42)

# Split the temporary set into validation and test sets
validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Print the sizes of each split
print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(validation_df)}")
print(f"Test set size: {len(test_df)}")

# Save the splits to CSV files
train_csv_path = "/content/drive/MyDrive/Master/Anul 1/BioNLP/dataset/train_health_content.csv"
validation_csv_path = "/content/drive/MyDrive/Master/Anul 1/BioNLP/dataset/validation_health_content.csv"
test_csv_path = "/content/drive/MyDrive/Master/Anul 1/BioNLP/dataset/test_health_content.csv"

train_df.to_csv(train_csv_path, index=False)
validation_df.to_csv(validation_csv_path, index=False)
test_df.to_csv(test_csv_path, index=False)

print(f"Training set saved to {train_csv_path}")
print(f"Validation set saved to {validation_csv_path}")
print(f"Test set saved to {test_csv_path}")

Training set size: 1342
Validation set size: 447
Test set size: 448
Training set saved to /content/drive/MyDrive/Master/Anul 1/BioNLP/dataset/train_health_content.csv
Validation set saved to /content/drive/MyDrive/Master/Anul 1/BioNLP/dataset/validation_health_content.csv
Test set saved to /content/drive/MyDrive/Master/Anul 1/BioNLP/dataset/test_health_content.csv


In [56]:
train_df

,id,title,text,rating,label
2093,story_reviews_01118,Acupuncture May Help Ease Hot Flashes Tied to ...,"En Español\n\nMONDAY, April 25, 2011 (HealthDa...",3,0
748,story_reviews_01350,Aspirin May Cut Prostate Cancer Deaths,The findings were released at a news briefing ...,3,0
1835,story_reviews_00749,Folic acid in pregnancy tied to lower autism risk,NEW YORK (Reuters Health) - Women who took ext...,4,0
925,story_reviews_01065,"Sleeptime Head-Cooling Cap Eases Insomnia, Stu...",By Jenifer Goodwin\n\nHealthDay Reporter\n\nTU...,4,0
167,news_reviews_00469,Chrono Therapeutics' Smoking Cessation Technol...,"CHICAGO, March 4, 2016 /PRNewswire/ -- Chrono ...",2,1
...,...,...,...,...,...
1638,story_reviews_00175,Immune-Focused Drug May Be New Weapon Against ...,"En Español\n\nMONDAY, Sept. 11, 2017 (HealthDa...",4,0
1095,story_reviews_00467,The Benefits of Intensive Cardiac Rehabilitati...,Despite considerable progress in prevention an...,3,0
1130,story_reviews_01529,Experimental flu treatment may help related virus,WASHINGTON (Reuters) - An experimental drug be...,3,0
1294,story_reviews_00892,Bypass Surgery Edges Stents For Heart Treatment,Bypass Surgery Edges Stents For Heart Treatmen...,4,0


In [57]:
train_label_counts = train_df['label'].value_counts()
print("Training set label counts:")
print(train_label_counts)

# Count the number of instances for each label in the validation set
validation_label_counts = validation_df['label'].value_counts()
print("\nValidation set label counts:")
print(validation_label_counts)

# Count the number of instances for each label in the test set
test_label_counts = test_df['label'].value_counts()
print("\nTest set label counts:")
print(test_label_counts)

Training set label counts:
label
0    896
1    446
Name: count, dtype: int64

Validation set label counts:
label
0    294
1    153
Name: count, dtype: int64

Test set label counts:
label
0    299
1    149
Name: count, dtype: int64


In [2]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [5]:
train_dataset['input'][0]

'Acupuncture May Help Ease Hot Flashes Tied to Prostate Cancer Treatment En Español\n\nMONDAY, April 25, 2011 (HealthDay News) -- Acupuncture might help reduce the hot flashes that frequently affect prostate cancer patients while they\'re on hormone therapy, a small study suggests.\n\nThe findings don\'t confirm that the ancient Chinese discipline relieves hot flashes, and only 14 men participated in the study, which was conducted by researchers at New York Methodist Hospital and Weill Cornell Medical College of Cornell University.\n\nStill, "our study shows that physicians and patients have an additional treatment for something that affects many men undergoing prostate cancer treatment and actually has long-term benefits, as opposed to more side effects," said lead author Dr. Hani Ashamalla, a radiation oncologist at New York Methodist Hospital, in a news release from the American Society for Radiation Oncology.\n\nAcupuncture, a popular form of alternative medicine, involves insertin

In [27]:
import re
train_csv_path = "/content/drive/MyDrive/Master/Anul 1/BioNLP/dataset/train_health_content.csv"
validation_csv_path = "/content/drive/MyDrive/Master/Anul 1/BioNLP/dataset/validation_health_content.csv"
test_csv_path = "/content/drive/MyDrive/Master/Anul 1/BioNLP/dataset/test_health_content.csv"

# Read the CSV files
train_df = pd.read_csv(train_csv_path)
validation_df = pd.read_csv(validation_csv_path)
test_df = pd.read_csv(test_csv_path)


def clean_text(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'\W', ' ', text)   # Remove special characters
    text = re.sub(r'\d+', '', text)   # Remove digits
    text = text.strip()
    return text


train_df['input'] = (train_df['title'] + ' ' + train_df['text']).apply(clean_text)
validation_df['input'] = (validation_df['title'] + ' ' + validation_df['text']).apply(clean_text)
test_df['input'] = (test_df['title'] + ' ' + test_df['text']).apply(clean_text)

In [28]:
# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['input'], truncation=True, max_length=512, padding='max_length')

In [35]:
train_dataset = Dataset.from_pandas(train_df[['input', 'label']])
validation_dataset = Dataset.from_pandas(validation_df[['input', 'label']])
test_dataset = Dataset.from_pandas(test_df[['input', 'label']])

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=False)
validation_dataset = validation_dataset.map(tokenize_function, batched=False)
test_dataset = test_dataset.map(tokenize_function, batched=False)

# Set the format to PyTorch tensors
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
validation_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/1342 [00:00<?, ? examples/s]

Map:   0%|          | 0/447 [00:00<?, ? examples/s]

Map:   0%|          | 0/448 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset[0])

In [16]:
! pip install transformers[torch]
! pip install accelerate

In [14]:
! pip install transformers

In [20]:
! pip show accelerate

Name: accelerate
Version: 0.31.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [21]:
! pip install transformers -U

In [44]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Device placement
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader
learning_rates = [2e-5]
batch_sizes = [4, 16]
num_epochs = 20
optimizers = [optim.AdamW]
weight_decays = [0.0, 0.01]
dropout_probs = [0.2, 0.3]
num_warmup_steps = [0, 100, 500]
num_attention_heads = [8, 12]

best_f1 = 0.0
best_params = {}

for lr in learning_rates:
    for batch_size in batch_sizes:
        for optimizer_class in optimizers:
            for weight_decay in weight_decays:
                for dropout_prob in dropout_probs:
                    for num_heads in num_attention_heads:
                        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                        validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size)
                        test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

                        model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)
                        optimizer = optimizer_class(model.parameters(), lr=lr, weight_decay=weight_decay)
                        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
                        print(f"{lr}, {batch_size}, {optimizer_class}, {weight_decay}, {dropout_prob}, {num_heads}")
                        for epoch in range(num_epochs):
                            model.train()
                            for batch in train_dataloader:
                                input_ids = batch['input_ids'].to(device)
                                attention_mask = batch['attention_mask'].to(device)
                                labels = batch['label'].to(device)

                                optimizer.zero_grad()
                                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                                loss = outputs.loss
                                loss.backward()
                                optimizer.step()

                            model.eval()
                            predictions = []
                            true_labels = []
                            with torch.no_grad():
                                for batch in validation_dataloader:
                                    input_ids = batch['input_ids'].to(device)
                                    attention_mask = batch['attention_mask'].to(device)
                                    labels = batch['label'].to(device)

                                    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                                    logits = outputs.logits
                                    predictions.extend(torch.argmax(logits, axis=1).cpu().numpy())
                                    true_labels.extend(labels.cpu().numpy())

                            accuracy = accuracy_score(true_labels, predictions)
                            precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')

                            print(f"Epoch {epoch + 1}:")
                            print(f"  Validation Accuracy: {accuracy}")
                            print(f"  Validation Precision: {precision}")
                            print(f"  Validation Recall: {recall}")
                            print(f"  Validation F1: {f1}")

                            if accuracy > best_f1:
                                best_f1 = accuracy
                                best_params = {
                                    'learning_rate': lr,
                                    'batch_size': batch_size,
                                    'optimizer': optimizer_class.__name__,
                                    'weight_decay': weight_decay,
                                    'dropout_prob': dropout_prob,
                                    'num_attention_heads': num_heads,
                                    'epoch': epoch + 1
                                }

                            scheduler.step()

print(f"Best F1 Score: {best_f1}")
print("Best Hyperparameters:")
print(best_params)

In [ ]:
# 2e-05, 4, <class 'torch.optim.adamw.AdamW'>, 0.01, 0.2, 12
# Validation Accuracy: 0.7472035794183445
# 2e-05, 16, <class 'torch.optim.adamw.AdamW'>, 0.0, 0.3, 8

In [ ]:
# Test Accuracy: 0.6808035714285714
# Test Precision: 0.5189873417721519
# Test Recall: 0.5503355704697986
# Test F1: 0.5342019543973942
# num_epochs = 20
# lr = 2e-05
# optimizer = optim.Adam

In [53]:
from torch.utils.data import DataLoader
batch_size = 4

# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

num_epochs = 20
lr = 2e-05
optimizer = optim.Adam
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)
optimizer = optimizer(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
for epoch in range(num_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)


        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    predictions = []
    labels = []
    with torch.no_grad():
        for batch in validation_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels.extend(batch['label'].numpy())

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions.extend(torch.argmax(logits, axis=1).cpu().numpy())

    # Compute metrics
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    print(predictions)
    print(f"Epoch {epoch + 1}:")
    print(f"  Validation Accuracy: {accuracy}")
    print(f"  Validation Precision: {precision}")
    print(f"  Validation Recall: {recall}")
    print(f"  Validation F1: {f1}")

    # Adjust learning rate
    scheduler.step()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [52]:
model.eval()
test_predictions = []
test_labels = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        test_labels.extend(batch['label'].numpy())

        outputs =  model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        test_predictions.extend(torch.argmax(logits, axis=1).cpu().numpy())

test_accuracy = accuracy_score(test_labels, test_predictions)
test_precision, test_recall, test_f1, _ = precision_recall_fscore_support(test_labels, test_predictions, average='binary')

print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Test F1: {test_f1}")


Test Accuracy: 0.6808035714285714
Test Precision: 0.5189873417721519
Test Recall: 0.5503355704697986
Test F1: 0.5342019543973942
